<a href="https://colab.research.google.com/github/shreyasudaya/mathunlearn/blob/master/CIFAR10_Resnet_08_08_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import os

In [2]:
# Data augmentation and normalization
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [3]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [4]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)


Files already downloaded and verified


In [5]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified


In [6]:
# Load the pretrained ResNet18 model with the updated parameter
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(512, 10)  # Modify the final layer for CIFAR-10

In [7]:
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)

In [9]:
# Evaluate the model on the test dataset
def evaluate(loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the loaded model on the CIFAR-10 test images: %.2f %%' % (100 * correct / total))
    return 100 * correct / total

In [10]:
# Training the model
def train(epoch):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

In [11]:
# Training for 100 epochs
num_epochs = 10
best_acc = 0
save_path = './cifar10_resnet18_08_08_24.pth'

In [12]:
for epoch in range(num_epochs):
    train(epoch)
    scheduler.step()
    acc = evaluate(testloader)
    if acc > best_acc:
        best_acc = acc
        print(f'Saving model with accuracy {best_acc:.2f}%')
        torch.save(model.state_dict(), save_path)

[1,   100] loss: 1.329
[1,   200] loss: 0.994
[1,   300] loss: 0.882
Accuracy of the loaded model on the CIFAR-10 test images: 71.05 %
Saving model with accuracy 71.05%
[2,   100] loss: 0.780
[2,   200] loss: 0.731
[2,   300] loss: 0.729
Accuracy of the loaded model on the CIFAR-10 test images: 75.13 %
Saving model with accuracy 75.13%
[3,   100] loss: 0.668
[3,   200] loss: 0.663
[3,   300] loss: 0.641
Accuracy of the loaded model on the CIFAR-10 test images: 77.58 %
Saving model with accuracy 77.58%
[4,   100] loss: 0.599
[4,   200] loss: 0.598
[4,   300] loss: 0.583
Accuracy of the loaded model on the CIFAR-10 test images: 79.31 %
Saving model with accuracy 79.31%
[5,   100] loss: 0.550
[5,   200] loss: 0.564
[5,   300] loss: 0.548
Accuracy of the loaded model on the CIFAR-10 test images: 81.33 %
Saving model with accuracy 81.33%
[6,   100] loss: 0.516
[6,   200] loss: 0.518
[6,   300] loss: 0.510
Accuracy of the loaded model on the CIFAR-10 test images: 80.85 %
[7,   100] loss: 0.4

In [13]:
print('Finished Training')
print(f'Best Accuracy: {best_acc:.2f}%')

Finished Training
Best Accuracy: 83.33%


In [34]:
import numpy as np
file_path='/content/cifar10_resnet18_08_08_24.pth'
state_dict = torch.load(file_path, weights_only=True)

# Initialize an empty list to store the weights
weights_list = []

# Iterate over each parameter in the state dictionary
for param_name, param_tensor in state_dict.items():
    if isinstance(param_tensor, torch.Tensor):   # Ensure it's a tensor
        param_data = param_tensor.cpu().numpy()  # Move to CPU and convert to NumPy
        weights_list.append(param_data)

print(weights_list)

[array([[[[-9.77470074e-03,  1.63173233e-03, -1.51021685e-02, ...,
           5.13883010e-02,  3.22095007e-02, -2.55698748e-02],
         [ 3.27800550e-02,  3.99738476e-02, -9.54137966e-02, ...,
          -2.59567201e-01, -1.00876316e-01,  2.44177971e-02],
         [-2.65821777e-02,  7.96804726e-02,  3.35715026e-01, ...,
           5.56309819e-01,  2.90416986e-01,  7.23003596e-02],
         ...,
         [-1.73380773e-03,  5.77984154e-02,  8.06580856e-02, ...,
          -3.26841205e-01, -4.02433485e-01, -2.60192961e-01],
         [ 2.16718670e-02,  6.78803623e-02,  5.74812554e-02, ...,
           4.11018699e-01,  3.77543926e-01,  1.55666843e-01],
         [-2.23458577e-02,  1.46070812e-02, -1.87211484e-02, ...,
          -1.59563005e-01, -9.84075963e-02, -2.92294398e-02]],

        [[-2.38767434e-02, -2.63936576e-02, -4.93878983e-02, ...,
           2.20712293e-02,  1.18933581e-02, -4.23531644e-02],
         [ 7.13137016e-02,  6.36399910e-02, -8.96844342e-02, ...,
          -3.06597680

In [19]:
# Evaluate the model
evaluate(testloader)

Accuracy of the loaded model on the CIFAR-10 test images: 83.33 %


83.33

In [20]:
# Evaluate the model
evaluate(trainloader)

Accuracy of the loaded model on the CIFAR-10 test images: 86.53 %


86.534

In [21]:
# Load the .pth file (adjust the path to your file)
checkpoint = torch.load('./cifar10_resnet18_08_08_24.pth')

# Check the keys stored in the checkpoint
print(checkpoint.keys())


odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.num_batches_tracked', 'layer2.0.conv1.weight', 'layer2.0.bn1.weight', 'layer2.0.bn1.bias', 'layer2.0.bn1.running_mean', 'layer2.0.bn1.running_var', 'layer2.0.bn1.num_batches_tracked', 'layer2.0.conv2.weight', 'layer2.0.bn2.weight', 'layer2.0.bn2.bias', '

<ipython-input-21-be91f4661030>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('./cifar10_resnet18_08_08_24.pth')
